In [1]:
#Import Libraries

In [1]:
import torchvision
import torch
import torchvision.transforms as transforms
import os
import matplotlib.pyplot as plt
import numpy as np
import scipy
import cv2
from torch.utils.data import DataLoader
from torchvision import datasets
import pycocotools
from torchvision.datasets import CocoDetection
from pycocotools.coco import COCO
from matplotlib.patches import Rectangle
import random
from PIL import Image
from ultralytics import YOLO
import cvzone
import math
import torchvision.models.detection as detection

In [3]:
#Load the Data

In [2]:
train_path = 'Dataset/10_images/' #this is the train images path
val_path = 'Dataset/val2017/val2017/' #this is the validatino images path

In [3]:
transform = transforms.Compose([
    transforms.Resize((512, 512)), #resizes all the images to 512 by 512
    transforms.ToTensor() #converts the images to torch.tensor
])

In [5]:
train_dataset = CocoDetection(root=train_path, #the train path
                              annFile='Dataset/annotations_trainval2017/annotations/instances_train2017.json', #annotation file
                              transform=transform) #transforms the images

loading annotations into memory...
Done (t=22.41s)
creating index...
index created!


In [8]:
test_dataset = CocoDetection(root='val_path', #the validation path
                            annFile='Dataset/annotations_trainval2017/annotations/instances_val2017.json', #annotation file
                            transform=transform) #transforms the images

loading annotations into memory...
Done (t=0.83s)
creating index...
index created!


In [7]:
train_loader = DataLoader(
    train_dataset,
    batch_size=1,
    shuffle=True,
    num_workers = 8,
    pin_memory=True,
    drop_last=True
)

In [9]:
test_loader = DataLoader(
    test_dataset, 
    batch_size=32,
    shuffle=False,
    num_workers = 8,
    pin_memory=True,
    drop_last=True,
)

In [8]:
model = detection.fasterrcnn_resnet50_fpn(pretrained=True)

C:\Users\Shashwat\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Shashwat\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
num_classes = 80

In [10]:
in_features = model.roi_heads.box_predictor.cls_score.in_features

In [11]:
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

In [12]:
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [13]:
import torch.optim as optim

In [14]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [15]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device('cpu')

In [19]:
model.to(device)

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [16]:
epochs = 10

In [17]:
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, targets in train_loader:
        images = list(images.to(device)for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        optimizer.zero_grad()
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        losses.backward()
        optimizer.step()
        running_loss += losses.item()
    print(f"Epoch [{epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "C:\Users\Shashwat\anaconda3\Lib\site-packages\torch\utils\data\_utils\worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Shashwat\anaconda3\Lib\site-packages\torch\utils\data\_utils\fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Shashwat\anaconda3\Lib\site-packages\torch\utils\data\_utils\fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "C:\Users\Shashwat\anaconda3\Lib\site-packages\torchvision\datasets\coco.py", line 53, in __getitem__
    image = self._load_image(id)
            ^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Shashwat\anaconda3\Lib\site-